<a href="https://colab.research.google.com/github/flatlandLucaDando/Flatland/blob/flatland_v_3_deterministic/Flatland_Railways.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction 

---

This is a Workbook to execute the Flatland-RealWorld 3.06 code on colab

First thing is important to import some libraries


In [ ]:
import sys

if 'google.colab' in sys.modules:
    # Clone GitHub repository
    !git clone --single-branch --branch flatland_v_3_deterministic https://github.com/flatlandLucaDando/Flatland.git
    # Copy files required to run the code
    !pip install flatland-rl

    # Restart Runtime
    import os
    os.kill(os.getpid(), 9)

Cloning into 'Flatland'...
remote: Enumerating objects: 1403, done.
remote: Counting objects: 100% (1403/1403), done.
remote: Compressing objects: 100% (711/711), done.
remote: Total 1403 (delta 751), reused 1320 (delta 675), pack-reused 0
Receiving objects: 100% (1403/1403), 3.42 MiB | 4.81 MiB/s, done.
Resolving deltas: 100% (751/751), done.
     |████████████████████████████████| 2.2 MB 5.4 MB/s 
     |████████████████████████████████| 85 kB 4.3 MB/s 
     |████████████████████████████████| 1.6 MB 49.2 MB/s 
     |████████████████████████████████| 231 kB 42.9 MB/s 
     |████████████████████████████████| 245 kB 49.6 MB/s 
  Using cached pytest_runner-5.3.1-py3-none-any.whl (7.1 kB)
     |████████████████████████████████| 246 kB 49.5 MB/s 
     |████████████████████████████████| 1.0 MB 54.0 MB/s 
     |████████████████████████████████| 197 kB 49.7 MB/s 
     |████████████████████████████████| 119 kB 13.8 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████

In [1]:
import time
import numpy as np
import os
from datetime import datetime
import random
import sys
from argparse import ArgumentParser, Namespace
from pathlib import Path
from pprint import pprint

import psutil
from torch.utils.tensorboard import SummaryWriter
import torch
from typing import Callable, Tuple, Optional, Dict, List
from numpy.random.mtrand import RandomState

# Structure Rail
Contein the basilar structure of the rail, with the high velocity rails and the right, left, nord or sud rails

With i_flag you can select the different rails configurations

In [12]:
i = 3

if i == 1:
	# One rail, so no right or left rails  
	right_rails = [(0,0)]
	left_rails = [(0,0)]
	down_rails = [(0,0)]
	up_rails = [(0,0)]

	av_line = [(0,0)]

if i ==2:
	# Rails where the direction is right
	right_rails = []
	for i in range(8):
		right_rails.append((5,i+6))
	# Rails where the direction is left
	left_rails = []
	for i in range(8):
		left_rails.append((6,i+6))

	down_rails = [(0,0)]
	up_rails = [(0,0)]

	av_line = [(0,0)]

if i == 3:
	# One rail, so no right or left rails  
	right_rails = [(0,0)]
	left_rails = [(0,0)]
	down_rails = [(0,0)]
	up_rails = [(0,0)]

	av_line = [(0,0)]

# Data Structures

## Convoy


In [18]:
from enum import Enum
import itertools

# Type of a convoy, can be high velocity or regional
# The velocity are given by default depending on the type of convoy, 360 for HV, 180 for IC, 120 for Regional
class Type_of_convoy(Enum):
	HIGH_VELOCITY = 1
	INTERCITY = 2
	REGIONAL = 3

# A convoy is a locomotive + wagons.
class Convoy:

	# For the id
	id_iter = itertools.count()

	def __init__(self, train_type, schedule = []):

		# identifier of the train
		self.id = next(Convoy.id_iter)
		# type of train (High velocity, Intercity, regional)
		self.train_type = train_type
		# schedule of the train
		self.schedule = []

		if train_type == Type_of_convoy.HIGH_VELOCITY:
			self.maximum_velocity = 1
		if train_type == Type_of_convoy.INTERCITY:
			self.maximum_velocity = 1/2
		if train_type == Type_of_convoy.REGIONAL:
			self.maximum_velocity = 1/3

	def add_train_run(self, train_run):
		self.schedule.append(train_run)

	# Discover the starting time of a certain run
	def starting_time(self, run_number):
		return self.schedule[run_number][0]

	# Convert velocity (maximum possible velocity is 360)
	def velocity_conversion(self):
		print(self.maximum_velocity * 360)

	# Verificate that a schedule is possible (if someone want to write manually)
	def schedule_verification(self, schedule, num_trains_run):
		if type(schedule) == list:
			row_num = len(schedule)
			column_num = len(schedule[0])
			if row_num >  1:
				for num_of_runs in range(num_trains_run - 1):
					for steps in range(len(schedule) - 1):
						if schedule[num_of_runs][step + 1] <= schedule[num_of_runs][step]:
							print('==========================================================')
							print('The time to connect stations',step,'and',step + 1,'have to be > 0')

			else:
				for steps in range(len(schedule) - 1):
					if schedule[num_of_runs][step + 1] <= schedule[num_of_runs][step]:
						print('==========================================================')
						print('The time to connect stations',step,'and',step + 1,'have to be > 0')
		else:
			print('A schedule should comprend different stations')
		return True


## Line

In [14]:
import itertools

# A line is considered from a city to another, tipically joining several cities
class Line:

	# For the id
	id_iter = itertools.count()

	def __init__(self, type_line, stations, stops):
		# ID of the line
		self.id = next(Line.id_iter)
		# type of line (High velocity or regional)
		self.type_line = type_line
		# Stations where the line pass from
		self.stations = stations
		# Stops are stations where the train have to stop
		# Is an array with 0 where not stop and 1 where train stops
		self.stops = stops

		if type(stations) == int or type(stops) == int:
			print('The stations of a line should be more than one, and the dimension of the stops should be the same of the stations')
		else:
			if(len(stations)) != (len(stops)):
				print('Stations and Stops have to be the same lenght')


	def inversion_of_line(self):
		self.direction = self.direction * -1

## Rail Connection

In [15]:
from enum import Enum
import itertools

# A connection can be: High velocity or normal
# The velocity are given by default depending on the type of connection, 360 for HV and 120 for normal
class Connection_type(Enum):
	HIGH_VELOCITY_RAIL = 1
	NORMAL_RAIL = 2


# A physical connection between two stations
class Rail_connection:

	# For the id
	id_iter = itertools.count()

	def __init__(self, station_a, station_b, rail_connection_type, additional_runtime_percent, max_speed_usable: int = 1/2):
		# each railway section has an id
		self.id = next(Rail_connection.id_iter)
		# Station A and B are the two connected stations
		self.station_a = station_a
		self.station_b = station_b
		# A connection can be: High velocity or normal
		self.rail_connection_type = rail_connection_type
		# Maximum speed usable in the rails
		self.max_speed_usable = max_speed_usable
		# Additional Runtime Percent is the percent [0-1] of the min run time that is added to the min run time, if the train is on schedule.
		# In general, he actual run time is computed as min run time + max(0, (min run time*additionalRuntimePercent)-actual_delay).
		self.additional_runtime_percent = additional_runtime_percent

		if rail_connection_type == Connection_type.HIGH_VELOCITY_RAIL:
			self.max_speed_usable = 1
		if rail_connection_type == Connection_type.NORMAL_RAIL:
			self.max_speed_usable = 1 / 2

## Station

In [16]:
from flatland.core.grid.grid_utils import Vec2dOperations as Vec2d

# Station
class Station:

	# TODO metti valori precisi per min wait time (lis)
	def __init__(self, name, position, capacity, min_wait_time, additional_wait_percent, importance, railway_topology):
		# Name of the station (e.g. Milano, Torino etc etc)
		self.name = name
		# Position (y,x) of the station in the railway
		self.position = position
		# Capacity of the station, num of rails in the station
		self.capacity = capacity
		# Minimum wait time for the trains, a train can't stop less than the min wait time
		self.min_wait_time = min_wait_time
		# Additional wait percent is the percent [0-1] of the minWaitTime that is added to the minWait at each stop, if the train is on schedule. 
		# In general, the actual (runtime) stopping time is computed as minWaitTime + max(0, (minWaitTime*additionalWaitTimePercent)-actual_delay).
		self.additional_wait_percent = additional_wait_percent
		# Stations have different importance depending on how much they are big and how much people they transport depending on the time
		self.importance = importance
		# Rails of the station
		self.rails = (0,0)
		
		return
		'''
		if capacity == 1:
			self.rails = None
		else:
			self.rails = self.calculate_rails(railway_topology)
		return
		'''

	def time_in_station(self, train_velocity):
		# The len of the rails is given by the station
		len_rails = len(self.rails_in_station[0])
		# The time needed is given by the formula (len * 1/velocity + waiting time + 10% of time)
		time_needed =  len_rails * int(pow(train_velocity, -1)) + self.min_wait_time[0]
		time_needed += int(time_needed/10)
		return time_needed

	def calculate_rails(self, railway_topology):
		# Number of rails of the station
		num_of_rails = self.capacity
		center_of_station = self.position 
		rail_shape = railway_topology.grid.shape

		#Flag
		right = False  # Flag to understand where to go right or left
		north = True    # Flag to understand where to go right or left

		# Indicating the incrementing number north or sud (in case of horizontal station), east ovest (in case of vertical stations)
		difference_from_original = 0 

		# Counter
		counter_of_rails = 0
		# Rails of the station, has the position of the rails. Each row is a rail
		self.rails = []
		# Contein the single rail positions
		single_rail_in_station = []

		# Counter to check the station is well positioned, to avoid the while goes for eternity
		counter = 0

		# Starting position the center of station
		current_position = (self.position[0], self.position[1])

		while counter_of_rails < num_of_rails:

			counter += 1

			if counter > 500:
				raise ImportError('The position of the station, or the capacity should be different, check for the right position or capacity, cant calculate the rails')

			# Horizontal rail
			if railway_topology.grid[current_position] == 1025:
				# The starting rail is a rail of the station
				single_rail_in_station.append(current_position)
				# Going to left
				if not right:
					new_pos = (0,-1)
				# Going to right
				if right:
					new_pos = (0, 1)
				# Update the position to left or right
				current_position = Vec2d.add(current_position, new_pos)

				# is position inside the grid?
				if current_position[0] >= rail_shape[0] or current_position[0] < 0 or current_position[1] >= rail_shape[1] or current_position[1] < 0:
					continue 
				# Current position != 0 and going left?
				# Starting going to left
				if railway_topology.grid[current_position] != 0 and not right:
					# If not 0 the rail is in the station
					single_rail_in_station.append(current_position)
					# I'm arrived at the end of the rail in station
					if railway_topology.grid[current_position] != 1025:
						# The entrance is on left
						if current_position[0] == self.position[0]:
							self.station_entrance = current_position
						# Restart the position to the original once
						# If I'm north or sud the new original position is north or sud with respect to the real original
						if difference_from_original == 0:
							current_position = (self.position[0], self.position[1])
						else:
							current_position = (self.position[0] + difference_from_original, self.position[1])
						# Then going to right
						right = True
						continue
					continue

				# Current position != 0 and going right?	            		
				if railway_topology.grid[current_position] != 0 and right:
					# If not 0 the rail is in the station
					single_rail_in_station.append(current_position)
					# I'm arrived at the end of the rail in station
					if railway_topology.grid[current_position] != 1025:
						# The exit is on right
						if current_position[0] == self.position[0]:
							self.station_exit = current_position
						self.rails.append(single_rail_in_station)
						# Restart the position from original once
						current_position = (self.position[0], self.position[1])
						# A rail is ended
						counter_of_rails += 1
						# Resetting the flag
						right = False
						if north:
							# Going to north, checking if at north there is a rail or not 
							new_pos = (-1, 0)
							position = Vec2d.add(current_position, new_pos)
							if railway_topology.grid[position] != 0:
								# Checking if there is a rail
								current_position = Vec2d.add(current_position, new_pos)
								# Resetting the single rail
								single_rail_in_station = []
								# I'm one step north, so I'm one row up in the matrix
								difference_from_original -= 1
								continue
							else:
								# No rails upper, so I have to go down
								north = False
								# Restart the position from original once
								current_position = (self.position[0], self.position[1])
								# I'm at the original position, so difference is 0
								difference_from_original = 0
								continue
						else:
							# Going south
							new_pos = (+1, 0)
							position = Vec2d.add(current_position, new_pos)
							# Check if south I have a rail or not
							if railway_topology.grid[position] != 0:
								current_position = Vec2d.add(current_position, new_pos)
								# Resetting the single rail
								single_rail_in_station = []
								# I'm one step south, so I'm one row down in the matrix
								difference_from_original += 1
								continue
							else:
								# No rails down, so I have to go up
								north = True
								# Restart the position from original once
								current_position = (self.position[0], self.position[1])
								# I'm at the original position, so difference is 0
								difference_from_original = 0
								continue

			# Vertical rail same as for horizontal, but starting from south and north, and then left and right
			elif railway_topology.grid[current_position] == 32800:
				# The starting rail is a rail of the station
				single_rail_in_station.append(current_position)
				# Going down
				if not north:
					new_pos = (+1,0)
				# Going to right
				if north:
					new_pos = (-1, 0)
				# Update the position to left or right
				current_position = Vec2d.add(current_position, new_pos)

				# is position inside the grid?
				if current_position[0] >= rail_shape[0] or current_position[0] < 0 or current_position[1] >= rail_shape[1] or current_position[1] < 0:
					continue 
				# Current position != 0 and going left?
				# Starting going to left
				if railway_topology.grid[current_position] != 0 and not north:
					# If not 0 the rail is in the station
					single_rail_in_station.append(current_position)
					# I'm arrived at the end of the rail in station
					if railway_topology.grid[current_position] != 32800:
						# The entrance is on left
						if current_position[1] == self.position[1]:
							self.station_entrance = current_position
						# Restart the position to the original once
						# If I'm north or sud the new original position is north or sud with respect to the real original
						if difference_from_original == 0:
							current_position = (self.position[0], self.position[1])
						else:
							current_position = (self.position[0], self.position[1] + difference_from_original)
						# Then going to right
						north = True
						continue
					continue

				# Current position != 0 and going right?	            		
				if railway_topology.grid[current_position] != 0 and north:
					# If not 0 the rail is in the station
					single_rail_in_station.append(current_position)
					# I'm arrived at the end of the rail in station
					if railway_topology.grid[current_position] != 32800:
						# The exit is on right
						if current_position[1] == self.position[1]:
							self.station_exit = current_position
						self.rails.append(single_rail_in_station)
						# Restart the position from original once
						current_position = (self.position[0], self.position[1])
						# A rail is ended
						counter_of_rails += 1
						# Resetting the flag
						north = False
						if not right:
							# Going to left, checking if at north there is a rail or not 
							new_pos = (0, -1)
							position = Vec2d.add(current_position, new_pos)
							if railway_topology.grid[position] != 0:
								# Checking if there is a rail
								current_position = Vec2d.add(current_position, new_pos)
								# Resetting the single rail
								single_rail_in_station = []
								# I'm one step north, so I'm one row up in the matrix
								difference_from_original -= 1
								continue
							else:
								# No rails upper, so I have to go down
								right = True
								# Restart the position from original once
								current_position = (self.position[0], self.position[1])
								# I'm at the original position, so difference is 0
								difference_from_original = 0
								continue
						else:
							# Going south
							new_pos = (0, +1)
							position = Vec2d.add(current_position, new_pos)
							# Check if south I have a rail or not
							if railway_topology.grid[position] != 0:
								current_position = Vec2d.add(current_position, new_pos)
								# Resetting the single rail
								single_rail_in_station = []
								# I'm one step south, so I'm one row down in the matrix
								difference_from_original += 1
								continue
							else:
								# No rails down, so I have to go up
								right = False
								# Restart the position from original once
								current_position = (self.position[0], self.position[1])
								# I'm at the original position, so difference is 0
								difference_from_original = 0
								continue

		# Eliminate duplicates
		rails_position = []
		rails_position_single = []
		for i in range(len(self.rails)):
			for j in range(len(self.rails[i])):
				if self.rails[i][j] not in rails_position_single:
					rails_position_single.append(self.rails[i][j])
			rails_position.append(rails_position_single)
			rails_position_single = []
		self.rails = rails_position

## Train Run

In [17]:
# A train run is the run on a line for a train (e.g. Genova-Milano --- Milano-Genova)
# The train run consider each intermediate station the train has to pass between the two "principal" stations
class Train_run:

	def __init__(self, line_belongin, starting_time, from_depot: bool = False, to_depot: bool = False, inverse_train_direction: bool = False):
		# Line in which the train run is, so it contein informations about the stations to stop and etc etc
		self.line_belongin = line_belongin
		# Starting time of the run
		self.starting_time = starting_time
		# FromDepot indicates whether this run starts from a depot. Similar for ToDepot.
		self.from_depot = from_depot
		self.to_depot = to_depot
		# If inverse line direction 
		self.inverse_train_direction = inverse_train_direction



# Flatland Custom Rail Generator

--- 
A custom rail is a rail defined by a matrix. Each cell of the matrix define the type of cell in the 2-D grid of the flatland environment



## Different type of cell

<img src="https://i.imgur.com/ruiRuep.jpg" width="500"/> <img src="https://i.imgur.com/sABiSuc.jpg" width="500"/>

* This is the function to create a custom rail

In [2]:
"""Rail generators (infrastructure manager, "Infrastrukturbetreiber")."""
from typing import Callable, Tuple, Optional, Dict, List

from numpy.random.mtrand import RandomState

from flatland.core.grid.grid4 import Grid4TransitionsEnum
from flatland.core.grid.rail_env_grid import RailEnvTransitions
from flatland.core.transition_map import GridTransitionMap


RailGeneratorProduct = Tuple[GridTransitionMap, Optional[Dict]]
""" A rail generator returns a RailGenerator Product, which is just
    a GridTransitionMap followed by an (optional) dict/
"""

RailGenerator = Callable[[int, int, int, int], RailGeneratorProduct]

# Number of agents are given by timetables (the num of the rows), target stations are also given by timetable 
def rail_custom_generator(rail_spec, train_stations_position: list = None, timetable: list = None):

    """
    Utility to convert a rail given by manual specification as a map of tuples
    (cell_type, rotation), to a transition map with the correct 16-bit
    transitions specifications.

    Parameters
    ----------
    rail_spec : list of list of tuples
        List (rows) of lists (columns) of tuples, each specifying a rail_spec_of_cell for
        the RailEnv environment as (cell_type, rotation), with rotation being
        clock-wise and in [0, 90, 180, 270].

    Returns
    -------
    function
        Generator function that always returns a GridTransitionMap object with
        the matrix of correct 16-bit bitmaps for each rail_spec_of_cell.

    New features:
    -------------
        Train station poition: to define the position of the station
        Target station: Define the target of the different agents
        Timetable: timetable contein the intermediate station and the time at which pass across them
    """

    def custom_generator(width: int, height: int, num_agents: int, num_resets: int = 0,
                  np_random: RandomState = None) -> RailGenerator:

        # All the cities are oriented in the same way in my model
        city_orientations = 1

        # Taking the number of agents from timetable
        num_of_agents = len(timetable)

        # Taking the target stations from timetable
        target_stations = []

        for agent in range(num_agents):
            target_stations.append(timetable[agent][-1])
      
        rail_env_transitions = RailEnvTransitions()

        height = len(rail_spec)
        width = len(rail_spec[0])
        rail = GridTransitionMap(width=width, height=height, transitions=rail_env_transitions)

        for r in range(height):
            for c in range(width):
                rail_spec_of_cell = rail_spec[r][c]
                index_basic_type_of_cell_ = rail_spec_of_cell[0]
                rotation_cell_ = rail_spec_of_cell[1]
                if index_basic_type_of_cell_ < 0 or index_basic_type_of_cell_ >= len(rail_env_transitions.transitions):
                    print("ERROR - invalid rail_spec_of_cell type=", index_basic_type_of_cell_)
                    return []
                basic_type_of_cell_ = rail_env_transitions.transitions[index_basic_type_of_cell_]
                effective_transition_cell = rail_env_transitions.rotate_transition(basic_type_of_cell_, rotation_cell_)
                rail.set_transitions((r, c), effective_transition_cell)

        return rail,  {'agents_hints': {
            'num_agents': num_of_agents,            
            'city_positions': train_stations_position,
            'train_stations': train_stations_position,
            'city_orientations': city_orientations,
            'targets' : target_stations,
            'timetable' : timetable 
        }}

    return custom_generator


* these are the utilities for the environment


In [5]:
from flatland.core.env_observation_builder import ObservationBuilder
from flatland.envs.observations import TreeObsForRailEnv
from flatland.envs.predictions import ShortestPathPredictorForRailEnv
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_generators import rail_from_file
from flatland.envs.line_generators import line_from_file
from flatland.envs.rail_env import RailEnvActions
from flatland.envs.step_utils.states import TrainState


def load_flatland_environment_from_file(file_name: str,
                                        load_from_package: str = None,
                                        obs_builder_object: ObservationBuilder = None,
                                        record_steps = False,
                                        ) -> RailEnv:
    """
    Parameters
    ----------
    file_name : str
        The pickle file.
    load_from_package : str
        The python module to import from. Example: 'env_data.tests'
        This requires that there are `__init__.py` files in the folder structure we load the file from.
    obs_builder_object: ObservationBuilder
        The obs builder for the `RailEnv` that is created.


    Returns
    -------
    RailEnv
        The environment loaded from the pickle file.
    """
    if obs_builder_object is None:
        obs_builder_object = TreeObsForRailEnv(
            max_depth=2,
            predictor=ShortestPathPredictorForRailEnv(max_depth=10))
    environment = RailEnv(width=1, height=1, rail_generator=rail_from_file(file_name, load_from_package),
                          line_generator=line_from_file(file_name, load_from_package),
                          number_of_agents=1,
                          obs_builder_object=obs_builder_object,
                          record_steps=record_steps,
                          )
    return environment

# TODO riferisciti a un agente (EnvAgent)
# Usa le azioni dell'agente specifico
def delay_a_train(delay: int, delay_time: int, time_of_train_generation: int, actions, train):
    
    i_agent = train.handle
    train_velocity = train.speed_counter.speed

    
    actions_scheduled = [0] * (len(actions[i_agent]) + delay)
    
    # Copy the actions scheduled for the train before the delay
    for i in range(delay_time - time_of_train_generation):
        actions_scheduled[i] = actions[i_agent][i]
    # Delay the train (STOP)  
    for i in range(delay):
        if (i + delay_time - time_of_train_generation) < 0 or (delay_time - time_of_train_generation) > len(actions[i_agent]):
            raise ImportError('The train is not present in the environment, check the delay time')
        actions_scheduled[i + delay_time - time_of_train_generation] = RailEnvActions.STOP_MOVING
    # Copy the actions scheduled for the train after the delay
    for i in range(len(actions[i_agent]) - (delay_time - time_of_train_generation)):
        actions_scheduled[i + delay_time - time_of_train_generation + delay] = actions[i_agent][i + delay_time - time_of_train_generation]
    
    actions[i_agent] = actions_scheduled
    return 


# Function to convert decimal number to base number
def actions_decimal_to_base(base, number_to_convert, num_agents):
    division = number_to_convert
    result = []
    while division != 0:
        result.append(division % base)
        division = int(division / base)
    actions_to_perform = result[::-1]
    if len(actions_to_perform) < num_agents:
        zero_to_add = num_agents - len(actions_to_perform)
        for i in range(zero_to_add):
            actions_to_perform.insert(0,0)
    return actions_to_perform


def make_a_deterministic_interruption(agent_to_interrupt, interruption_time):
    state = agent_to_interrupt.state
    if state == TrainState.MOVING:
        agent_to_interrupt.malfunction_handler.malfunction_down_counter = interruption_time



# Custom schedule generator


In [10]:
"""Schedule generators (railway undertaking, "EVU")."""
from typing import Tuple, List, Callable, Mapping, Optional, Any

from enum import IntEnum
from numpy.random.mtrand import RandomState
 
from flatland.core.transition_map import GridTransitionMap
from flatland.envs.agent_utils import EnvAgent
from typing import List, NamedTuple, Callable

from flatland.core.grid.grid4 import Grid4TransitionsEnum
from flatland.core.grid.grid_utils import IntVector2DArray


Schedule = NamedTuple('Schedule', [('agent_positions', IntVector2DArray),
                                   ('agent_directions', List[Grid4TransitionsEnum]),
                                   ('agent_targets', IntVector2DArray),
                                   ('agent_speeds', List[float]),
                                   ('agent_malfunction_rates', List[int]),
                                   ('max_episode_steps', int)]) 

def check_rail_road_direction(rail: GridTransitionMap, timetable):
    # To establish the direction of trains in the railroas I define a simple law, as for the cars, each trains has to 
    # go the direction that let them to have the right free

    agents_direction = [0]*len(timetable)
    path_result = [0]*len(timetable)

    #print(rail.grid[7 ,13],rail.grid[7 ,12],rail.grid[7 ,11],rail.grid[7 ,10],rail.grid[7 ,9],rail.grid[7 ,8],rail.grid[7 ,7],rail.grid[7 ,6], rail.grid[7 ,5])
    
    for i in range (len(timetable)):
        # Consider the a_star result to calculate the direction
        path_result[i] = (a_star(rail,timetable[i][0][0],timetable[i][0][1]))
        if path_result[i] == []:
            agents_direction[i] = 1
            continue
        if len(path_result[i]) == 1:
            agents_direction[i] = 1
            continue
        difference_x = path_result[i][0][1] - path_result[i][1][1]
        difference_y = path_result[i][0][0] - path_result[i][1][0]
        if difference_y == 1:
            agents_direction[i] = 0
        if difference_x ==  -1:
            agents_direction[i] = 1
        if difference_y == -1:
            agents_direction[i] = 2
        if difference_x == 1:
            agents_direction[i] = 3

    return agents_direction

AgentPosition = Tuple[int, int]
ScheduleGenerator = Callable[[GridTransitionMap, int, Optional[Any], Optional[int]], Schedule]



class RailEnvActions(IntEnum):
	DO_NOTHING = 0  # implies change of direction in a dead-end!
	MOVE_LEFT = 1
	MOVE_FORWARD = 2
	MOVE_RIGHT = 3
	STOP_MOVING = 4
	REVERSE = 5

	@staticmethod
	def to_char(a: int):
		return {
			0: 'B',
			1: 'L',
			2: 'F',
			3: 'R',
			4: 'S',
			5: 'G'
		}[a]


def custom_schedule_generator(timetable, speed_ratio_map: Mapping[float, float] = None, seed: int = 1) -> ScheduleGenerator:

#class Custom_schedule_generator(BaseSchedGen):
	"""

	This is a custom schedule generator, create a schedule with the timetable, and the station where the trains should pass
	"""

	def generate_custom(rail: GridTransitionMap, num_agents: int, hints: Any = None, num_resets: int = 0,
				  np_random: RandomState = None) -> Schedule:
		"""

		The generator that assigns tasks to all the agents
		:param rail: Rail infrastructure given by the rail_generator
		:param num_agents: Number of agents to include in the schedule
		:param hints: Hints provided by the rail_generator These include positions of start/target positions
		:param num_resets: How often the generator has been reset.
		:return: Returns the generator to the rail constructor
		"""

		train_stations = hints['train_stations']
		city_positions = hints['city_positions']
		city_orientation = hints['city_orientations']
		max_num_agents = hints['num_agents']

		if num_agents > max_num_agents:
			num_agents = max_num_agents
			warnings.warn("Too many agents! Changes number of agents.")
		# Place agents and targets within available train stations
		agents_position = []
		agents_target = []
		agents_direction = []

		# Define the agent positions
		# TODO capisci se così va bene o no
		for agent_i in range (len(timetable)):
			agents_position.append(timetable[agent_i][0][0])
			agents_target.append(timetable[agent_i][0][-1])


		# Define the direction of the trains based on the rail they occupy
		# Input --> the topology of the network, the position of the trains
		# Output --> an array with the directions of the trains
		# DIRECTIONS: 0 = UP, 1 = RIGHT, 2 = DOWN, 3 = LEFT

		agents_direction = check_rail_road_direction(rail, timetable)


		_runtime_seed = seed + num_resets

		if speed_ratio_map:
			speeds = speed_initialization_helper(num_agents, speed_ratio_map, seed=_runtime_seed, np_random=np_random)
		else:
			speeds = [1.0] * len(agents_position)

		# We add multiply factors to the max number of time steps to simplify task in Flatland challenge.
		# These factors might change in the future.
		timedelay_factor = 4
		alpha = 2
		max_episode_steps = 1000

		#print(agents_position, agents_target, agents_direction)

		return Schedule(agent_positions=agents_position, agent_directions=agents_direction,
						agent_targets=agents_target, agent_speeds=speeds, agent_malfunction_rates=None,
						max_episode_steps=max_episode_steps)

	return generate_custom  #(station_to_traverse = [(21, 37), (15, 51)])

* These are the plan to follow utils
*testo in corsivo*

In [20]:
# import chain
from itertools import chain
from enum import IntEnum
import random
# This is to test if the timetable is valid or not
from flatland.core.grid.grid4_astar import a_star

class RailEnvActions(IntEnum):
    DO_NOTHING = 0  # implies change of direction in a dead-end!
    MOVE_LEFT = 1
    MOVE_FORWARD = 2
    MOVE_RIGHT = 3
    STOP_MOVING = 4
    REVERSE = 5

    @staticmethod
    def to_char(a: int):
        return {
            0: 'B',
            1: 'L',
            2: 'F',
            3: 'R',
            4: 'S',
            5: 'G'
        }[a]

# Check if the timetable is feaseble or not
def control_timetable(timetable, railway_topology):
    # Check for all the trains
    for trains in range (len(timetable)):       
        # Check for all the stations
        # Calculate the difference of two different times, so i don't need the last term to cycle          
        for stations in range (len(timetable[trains][1]) - 1):   
            if (timetable[trains][1][stations] - timetable[trains][1][stations + 1]) >= 0:
                print('===================================================================================================================================')
                print('Attention!!! The agent number', trains, 'has a problem in the timetable, times to reach stations', stations, 'and', (stations+1), 'are not right')
                print('The time to reach the successive station SHOULD BE > 0, pay attenction to the timetable')
                # Function that check if the time to reach a station defined by the timetable are possible or not,
                # Return the time minimum time to reach two different stations depending on the distance and on the line type (high velocity, regional...)
            time_to_next_station = time_to_reach_next_station(timetable[trains][0][stations], timetable[trains][0][stations + 1], railway_topology, timetable, trains)
            # Control if the time to reach the next station is possible (considering maximum velocities of lines and the distances between two stations)
            if time_to_next_station > (timetable[trains][1][stations+1]- timetable[trains][1][stations]):
                print('===================================================================================================================================')
                print('Attention!!! Agent number', trains, 'has a problem in the timetable, times to reach stations', stations, 'and', (stations+1), 'are not right')
                print('The time to reach the next station SHOULD BE HIGHER. The minimum time to reach the station should be:', time_to_next_station)
    return

# TODO aggiungere dei controlli 
# - controllare che la posizione in cui si mette il goal del secondo treno sia possibile
# - controllare il tempo necessario nella stazione (in base anche alla rialzatura in cui si mette il treno 
#   e.g. treno spostato su di due caselle, tempo necessario + 4 * velocità alla meno uno)

def divide_trains_in_station_rails(timetable, railway_topology):
    # The number of different stations presented in the timetable
    different_stations = 0
    # The position of different stations presented in the timetable
    station_positions = []
    # Check how many different stations are in the timetable
    for i in range(len(timetable)): # for all the trains
        for k in range(len(timetable[i][0])): # for all the stations
            if timetable[i][0][k] not in station_positions:
                station_positions.append(timetable[i][0][k])
                different_stations += 1

    # Indexes contein the indexes of the station like this
    # 0 for the first station in station position 
    # 1 for the second and so on
    indexes = []
    # Local variable for the loop
    single_index = []

    for j in range(len(timetable)): # for all the trains
        single_index = [0]*len(timetable[j][0])
        for k in range(len(timetable[j][0])): # for all the stations
            for i in range(len(station_positions)): # for all the different stations discovered
                if station_positions[i] == timetable[j][0][k]:
                    single_index[k] = i
        indexes.append(single_index)
    
    # Stations have a capacity, if more trains with respect to the capacity
    # of the station are present there is a problem
    counter_of_trains = [0] * different_stations  # counter of trains in a certain station
    
    # TODO fai un ciclo for che cicli gli step nella tabella oraria per aggiornare il counter trains
    # quando più treni sono nella stessa stazione aumenta, quando un treno esce dalla stazione diminuisce
    # aggiungi una lettera che cicli sulle due stazioni	(ad ora le stazioni girano su tutta la timetable)
    # Calculating the maximum time the agents have to stay in env 
    
     # Here controll if two or more trains have to reach the same station at the same time
    for m in range(len(timetable)):  # for all the trains m
        for n in range(len(timetable)): # for all the trains n with m!=n
            for k in range(len(timetable[m][1])): # for all the stations of m
                for l in range(len(timetable[n][1])): # for all the stations of n
                    if m!=n and m<n:
                        # time at which the trains have to reach the stations
                        time_train_a = timetable[m][1][k]
                        time_train_b = timetable[n][1][l]
                        # if they have to reach the same stations
                        if indexes[m][k] == indexes[n][l]: 
                            threshold_time = calculate_time_in_station(timetable,m,n,k,l)
                            # if the time at which they have to reach the station is similar (calculated the time needed in the stations for the trains)
                            if time_train_a - time_train_b < threshold_time and time_train_a - time_train_b > -threshold_time:
                                # I change the goal of the convoy that first reach the station
                                timetable[n][0][l] = (station_positions[indexes[n][l]][0] - 1, station_positions[indexes[n][l]][1])
                            


# Define the scheduled actions the agents have to do
def action_to_do(timetable, railway_topology):
    # Path to do to arrive to the right station

    # L'idea vuole essere quella di avere un tempo necessario per stare in stazione
    # Se due treni nell'intorno di tempo devono stare nella stessa stazione bisogna indirizzarli 
    # in binari liberi diversi, quindi il passaggio diventa diverso.

    path_result = []
    # Calculate the path for all the trains
    for train_i in range (len(timetable)):
        # Number of stations in the train i
        num_of_stations = len(timetable[train_i][0])
        # The partial result a train run
        path_partial_result = []
        for station in range(num_of_stations - 1): 
            path_partial_result.append(a_star(railway_topology,timetable[train_i][0][station],timetable[train_i][0][station + 1]))
            if path_partial_result == []:
                raise ImportError('There s not a path between station', station, 'and station', station + 1 )

        # Final result for all the trains and train runs
        path_result.append(path_partial_result)

    # Calculate the actions that have to be done
    actions_to_do = []
    for train_i in range (len(timetable)):
        # Number of stations in the train i
        num_of_stations = len(timetable[train_i][0])
        # Flag that tells me that the next step is particular
        next = False
        # Each train occupy a row in the action_to_do matrix 
        actions_single_train = []
        # I need the direction of the last run for the reverse action
        direction_last_run = 0
        for station in range (num_of_stations - 1):
            # Each train occupy a row in the action_to_do matrix 
            actions_single_train_run = []
            for step in range (len(path_result[train_i][station])):
                # If i'm restarting from the final station of the train run, i have to wait till is the time to restart
                if len(path_result[train_i][station]) == 1:
                    time_to_wait = timetable[train_i][1][station + 1] - timetable[train_i][1][station]
                    for i in range(time_to_wait):
                        actions_single_train_run.append(RailEnvActions.STOP_MOVING)
                    continue
                # Calculate the direction of the trains at each step
                if step == 0:
                    difference_y = path_result[train_i][station][step][0] - path_result[train_i][station][step + 1][0]
                    difference_x = path_result[train_i][station][step][1] - path_result[train_i][station][step + 1][1]
                    if difference_y == 1:
                        direction = 0
                    if difference_x ==  -1:
                        direction = 1
                    if difference_y == -1:
                        direction = 2
                    if difference_x == 1:
                        direction = 3 
                else:
                    difference_y = path_result[train_i][station][step - 1][0] - path_result[train_i][station][step][0]
                    difference_x = path_result[train_i][station][step - 1][1] - path_result[train_i][station][step][1]
                    if difference_y == 1:
                        direction = 0
                    if difference_x ==  -1:
                        direction = 1
                    if difference_y == -1:
                        direction = 2
                    if difference_x == 1:
                        direction = 3 
                # Variable to count the number of possible path at each cell, is an int with the number of possible path
                if not step == 0:
                    # Specific case, a train is at the boarder of two different lines, 
                    # if this appen I have to consider the previous transition at the next time stamp due to the fact the velocity changes
                    if next:
                        multiple_path = railway_topology.get_transitions(path_result[train_i][station][step-1][0],path_result[train_i][station][step-1][1],prev_direction).count(1)
                        next = False
                    elif (path_result[train_i][station][step] in av_line) and not (path_result[train_i][station][step - 1] in av_line):
                        multiple_path = railway_topology.get_transitions(path_result[train_i][station][step][0],path_result[train_i][station][step][1],prev_direction).count(1)
                        next = True
                    else:
                        multiple_path = railway_topology.get_transitions(path_result[train_i][station][step-1][0],path_result[train_i][station][step-1][1],prev_direction).count(1)
                # Starting with a move forward direction for the train
                if step == 0:
                    #actions_single_train.append(RailEnvActions.MOVE_FORWARD)
                    prev_direction = direction
                # If I'm not at the start of the train 
                else:
                    # The direction doesn't change
                    if num_of_stations > 1 and station >= 1 and step <= 1:
                        if (direction - direction_last_run == 2 or direction - direction_last_run == -2):
                            # If I'm in an hig velocity line velocity is define only by the type of train
                            actions_single_train_run.append(RailEnvActions.REVERSE)
                            prev_direction = direction
                            continue

                    if direction - prev_direction == 0:
                        # If I'm in an hig velocity line velocity is define only by the type of train
                        if path_result[train_i][station][step - 1] in av_line:
                            velocity = timetable[train_i][2]
                        # If I'm in other line velocity is the minimum between 1/2 (the velocity of the line) and the type of train velocity
                        else:
                            velocity = min(timetable[train_i][2], 1/2)
                        for i in range(int(pow(velocity, -1))):
                            #print('Test per capire come varia',i, 'Treno numero', train_i)
                            actions_single_train_run.append(RailEnvActions.MOVE_FORWARD)
                        # I'm arrived at the station?
                        if step == (len(path_result[train_i][station]) - 1):
                            # If the next station is the last one of the train run I don't have to stop for the min wait time
                            if station != (num_of_stations - 2):
                                if len(path_result[train_i][station + 1]) == 1:
                                    continue
                            # If is an intermediate station I need to stop the min wait time
                            for i in range(3):   # TODO ADD THE WAITING TIMES OF THE STATIONS
                                actions_single_train_run.append(RailEnvActions.STOP_MOVING)

                        prev_direction = direction
                    # I have to reverse the train direction when I arrive at the ending station
                    elif ((direction - prev_direction) == -2) or ((direction - prev_direction) == 2):
                        # If I'm in an hig velocity line velocity is define only by the type of train
                        if path_result[train_i][station][step - 1] in av_line:
                            velocity = timetable[train_i][2]
                        else:
                            velocity = min(timetable[train_i][2], 1/2)
                        for i in range(int(pow(velocity, -1))):
                            actions_single_train_run.append(RailEnvActions.REVERSE)
                        prev_direction = direction
                    # I have to move to left 
                    # and I have more then one possible path, so I go left at the deviation
                    # Depending on the direction of march the results can be -1 or -3
                    elif ((direction - prev_direction == -1) and (multiple_path > 1)) or ((direction - prev_direction == +3)):
                        # If I'm in an hig velocity line velocity is define only by the type of train
                        if path_result[train_i][station][step - 1] in av_line:
                            velocity = timetable[train_i][2]
                        # If I'm in other line velocity is the minimum between 1/2 (the velocity of the line) and the type of train velocity
                        else:
                            velocity = min(timetable[train_i][2], 1/2)
                        for i in range(int(pow(velocity, -1))):
                            actions_single_train_run.append(RailEnvActions.MOVE_LEFT)
                        prev_direction = direction
                    # I have to move right 
                    # and I have more then one possible path, so I go left at the deviation 
                    # Depending on the direction of march the results can be +1 or -3
                    elif ((direction - prev_direction == 1) and (multiple_path > 1)) or ((direction - prev_direction == -3) ):
                        # If I'm in an hig velocity line velocity is define only by the type of train
                        if path_result[train_i][station][step - 1] in av_line:
                            velocity = timetable[train_i][2]
                        # If I'm in other line velocity is the minimum between 1/2 (the velocity of the line) and the type of train velocity
                        else:
                            velocity = min(timetable[train_i][2], 1/2)
                        for i in range(int(pow(velocity, -1))):
                            actions_single_train_run.append(RailEnvActions.MOVE_RIGHT)
                        prev_direction = direction
                    else:
                        if path_result[train_i][station][step - 1] in av_line:
                            velocity = timetable[train_i][2]
                        else:
                            velocity = min(timetable[train_i][2], 1/2)
                        for i in range(int(pow(velocity, -1))):
                            actions_single_train_run.append(RailEnvActions.MOVE_FORWARD)
                        # I'm arrived at the station?
                        if step == (len(path_result[train_i][station]) - 1):
                            for i in range(3):   # TODO ADD THE WAITING TIMES OF THE STATIONS
                                actions_single_train_run.append(RailEnvActions.STOP_MOVING)
                        prev_direction = direction
            direction_last_run = direction

            actions_single_train.append(actions_single_train_run)
 
        if isinstance(actions_single_train[0], list):
            len(actions_single_train)
            actions_single_train = list(chain.from_iterable(actions_single_train))
        actions_to_do.append(actions_single_train)

    return actions_to_do

# Calculate the time to reach the stations to understand if timetable is right
def time_to_reach_next_station(departure_station_position, arrival_station_position, railway_topology, schedule, train_number):
    # First thing check the distance between two stations 
    result = a_star(railway_topology, departure_station_position, arrival_station_position)
    # Maximum velocity a train can achieve
    train_velocity = schedule[train_number][2]

    lenght_path = len(result)  # distance between stations

    # Array when I put at each step the time needed to make the path
    # The total time is the sum of the numbers
    time_array = []
    # Check the at each step which train i am and which line im in
    for step in range(lenght_path):
        if (result[step]) in av_line:
            time_array.append(pow(train_velocity,-1))
        else:
            time_array.append(pow(min(train_velocity, 1/2), -1))
    time_needed = sum(time_array)

    #print((time_needed + int(time_needed/10))) DEBUG

    # Adding to the time a 10% to face with problems in case it's neaded
    return (time_needed + int(time_needed/10))

# TODO calculate the av_rails, in order to distinguish them
# TODO calculate the right,left,up,down rails, in order to distinguish them
# TODO understad if the velocities are realistic or not (360 km for high velocity, 180 and 120 is realistic or not?)

def calculate_timetable(convoys, railway_topology):
    # The timetable that should be returned
    timetable = []
    # For each convoy
    for convoy_i in range(len(convoys)):
        # For each train run defined
        single_convoy_schedule = convoys[convoy_i].schedule
        single_convoy_schedule_len = len(convoys[convoy_i].schedule)
        single_convoy = []
        for num_of_runs in range(single_convoy_schedule_len):
            # The single train run
            single_train_run = []
            # The number of station to pass
            num_of_stations = len(single_convoy_schedule[num_of_runs].line_belongin.stations)
            # The station to stop
            stations_to_stop_position = []
            # Direction not inverted?
            if not single_convoy_schedule[num_of_runs].inverse_train_direction:
                for i in range(num_of_stations):
                    # append the station position in the right order
                    stations_to_stop_position.append(single_convoy_schedule[num_of_runs].line_belongin.stations[i].position)
            # Direction inverdet?
            else:
                for i in range(num_of_stations):
                    # append the station position in inverted order
                    stations_to_stop_position.append(single_convoy_schedule[num_of_runs].line_belongin.stations[num_of_stations - 1 - i].position)
            # Adding the starting time
            single_train_run.append(single_convoy_schedule[num_of_runs].starting_time)

            for stations in range(num_of_stations -1):
                departure_station_position = stations_to_stop_position[stations]
                arrival_station_position = stations_to_stop_position[stations + 1]
                # First thing check the distance between two stations
                result = a_star(railway_topology, departure_station_position, arrival_station_position)
                # Maximum velocity a train can achieve
                train_velocity = convoys[convoy_i].maximum_velocity 

                lenght_path = len(result)  # distance between stations

                # Array when I put at each step the time needed to make the path
                # The total time is the sum of the numbers
                time_array = []

                # Check the at each step which train i am and which line im in
                # Train should be in the middle of two line type
                for step in range(lenght_path):
                    if (result[step]) in av_line:
                        time_array.append(pow(train_velocity,-1))
                    else:
                        time_array.append(pow(min(train_velocity, 1/2), -1))
                time_needed = sum(time_array)
                # Adding to the time a 10% to face with problems in case it's neaded
                time_needed = time_needed + int(time_needed/10)

                # Adding the precedence time 
                if stations != (num_of_stations - 1):
                    if len(single_train_run) == 1 and type(single_convoy_schedule[num_of_runs].line_belongin.stations[stations].min_wait_time) == int:
                        single_train_run.append(int(time_needed + single_train_run[0] + single_convoy_schedule[num_of_runs].line_belongin.stations[stations].min_wait_time))
                    else:
                        # sum of time needed, the precedence time and the waiting time at the station
                        single_train_run.append(int(time_needed + single_train_run[stations] + single_convoy_schedule[num_of_runs].line_belongin.stations[stations].min_wait_time[0]))

            single_convoy.append(stations_to_stop_position)
            single_convoy.append(single_train_run)
            
        timetable.append(single_convoy)

    # This is needed in order to obtein the timetable-standard structure
    final_timetable = [] # The final timetable
    timetable_single_convoy = [] # Timetable of a single convoy
    timetable_position_example = [] # Partial timetable with the position of the stations to pass
    timetable_time_example = []  # Partial timetable with the time at which reach the stations
    single_position_timetable = []  # position and time of a single train run
    single_time_timetable = []

    for i in range(len(timetable)):
        for j in range(len(timetable[i])):
            if j % 2 == 0:
                timetable_position_example.append(timetable[i][j])
            else:
                timetable_time_example.append(timetable[i][j])
        for k in range(len(timetable_position_example)):
            single_position_timetable += (timetable_position_example[k])
        for k in range(len(timetable_time_example)):
            single_time_timetable += timetable_time_example[k]
        # The standard timetable form is (positions, time, train velocity)
        timetable_single_convoy.append(single_position_timetable)
        timetable_single_convoy.append(single_time_timetable)
        timetable_single_convoy.append(convoys[i].maximum_velocity)
        # Final timetable
        final_timetable.append(timetable_single_convoy)
        # Restart the partial results
        single_position_timetable = []
        timetable_position_example = []
        single_time_timetable = []
        timetable_time_example = []
        timetable_single_convoy = []

    return final_timetable

def calculate_time_in_station(timetable,train_a,train_b,index_a,index_b):
    time_a = 15
    time_b = 15
    if index_a != 0:
        time_a = timetable[train_a][1][index_a] - timetable[train_a][1][index_a - 1] + 15
    if index_b != 0:
        time_b = timetable[train_b][1][index_b] - timetable[train_b][1][index_b - 1] + 15
    return max(time_a,time_b)

# Rail Environment

## Agent

In [23]:
from flatland.envs.rail_env import RailEnvActions

# Import your own Agent or use RLlib to train agents on Flatland
# As an example we use a random agent instead
class RandomAgent:

	def __init__(self, state_size, action_size):
		self.state_size = state_size
		self.action_size = action_size


	# HERE DEFINE THE ACTIONS TO DO IN CASE THE AGENT IS NOT IN THE DETERMINISTIC PART 
	# For now the agents can only move forward (for DEBUG)
	def act(self, state):
		return RailEnvActions.MOVE_FORWARD

	def step(self, memories):
		"""

		Step function to improve agent by adjusting policy given the observations

		:param memories: SARS Tuple to be
		:return:
		"""

		return

	def save(self, filename):
		# Store the current policy
		return

	def load(self, filename):
		# Load a policy
		return

## Environment


### Rewards and Penalities


Here define the penalities and rewards for the agents

In [26]:
# Penalities 
step_penality = - 0.1      # a step is time passing, so a penality for each step is needed
stop_penality = - 0.1      # penality for stopping a moving agent
reverse_penality = - 0.1   # penality for reversing the march of an agent

target_reward = 100         # reward for an agent reaching his final target



### Training Flag


In [27]:
example_training = 'training1'

In [28]:
"""
Definition of the RailEnv environment.
"""
import random

from typing import List, Optional, Dict, Tuple

import numpy as np
from gym.utils import seeding

from flatland.utils.rendertools import RenderTool, AgentRenderVariant
from flatland.core.env import Environment
from flatland.core.env_observation_builder import ObservationBuilder
from flatland.core.grid.grid4 import Grid4Transitions
from flatland.core.transition_map import GridTransitionMap
from flatland.envs.agent_utils import EnvAgent
from flatland.envs.distance_map import DistanceMap
from flatland.envs.rail_env_action import RailEnvActions

from flatland.envs import malfunction_generators as mal_gen
from flatland.envs import rail_generators as rail_gen
from flatland.envs import line_generators as line_gen
from flatland.envs.timetable_generators import timetable_generator
from flatland.envs import persistence
from flatland.envs import agent_chains as ac

from flatland.envs.observations import GlobalObsForRailEnv

from flatland.envs.timetable_generators import timetable_generator
from flatland.envs.step_utils.states import TrainState, StateTransitionSignals
from flatland.envs.step_utils.transition_utils import check_valid_action
from flatland.envs.step_utils import action_preprocessing
from flatland.envs.step_utils import env_utils

class RailEnv(Environment):
    """
    RailEnv environment class.

    RailEnv is an environment inspired by a (simplified version of) a rail
    network, in which agents (trains) have to navigate to their target
    locations in the shortest time possible, while at the same time cooperating
    to avoid bottlenecks.

    The valid actions in the environment are:

     -   0: do nothing (continue moving or stay still)
     -   1: turn left at switch and move to the next cell; if the agent was not moving, movement is started
     -   2: move to the next cell in front of the agent; if the agent was not moving, movement is started
     -   3: turn right at switch and move to the next cell; if the agent was not moving, movement is started
     -   4: stop moving
     -   5: invert the direction of march

    Moving forward in a dead-end cell makes the agent turn 180 degrees and step
    to the cell it came from.

    The actions of the agents are executed in order of their handle to prevent
    deadlocks and to allow them to learn relative priorities.

    Reward Function:

    It costs each agent a step_penalty for every time-step taken in the environment. Independent of the movement
    of the agent. Currently all other penalties such as penalty for stopping, starting and invalid actions are set to 0.

    alpha = 1
    beta = 1
    Reward function parameters:

    - invalid_action_penalty = 0
    - step_penalty = -alpha
    - global_reward = beta
    - epsilon = avoid rounding errors
    - stop_penalty = 0  # penalty for stopping a moving agent
    - start_penalty = 0  # penalty for starting a stopped agent

    Stochastic malfunctioning of trains:
    Trains in RailEnv can malfunction if they are halted too often (either by their own choice or because an invalid
    action or cell is selected.

    Every time an agent stops, an agent has a certain probability of malfunctioning. Malfunctions of trains follow a
    poisson process with a certain rate. Not all trains will be affected by malfunctions during episodes to keep
    complexity managable.

    TODO: currently, the parameters that control the stochasticity of the environment are hard-coded in init().
    For Round 2, they will be passed to the constructor as arguments, to allow for more flexibility.

    """
    # Epsilon to avoid rounding errors
    epsilon = 0.01
    # NEW : REW: Sparse Reward
    alpha = 0.05
    beta = 1
    step_penalty = -1 * alpha
    global_reward = 1 * beta
    invalid_action_penalty = 0  # previously -2; GIACOMO: we decided that invalid actions will carry no penalty
    stop_penalty = -0.5  # penalty for stopping a moving agent
    reverse_penality = -0.5
    start_penalty = 0  # penalty for starting a stopped agent
    cancellation_factor = 1
    cancellation_time_buffer = 0

    def __init__(self,
                 width,
                 height,
                 max_episode_steps,
                 rail_generator=None,
                 line_generator=None,  # : line_gen.LineGenerator = line_gen.random_line_generator(),
                 number_of_agents=2,
                 obs_builder_object: ObservationBuilder = GlobalObsForRailEnv(),
                 malfunction_generator_and_process_data=None,  # mal_gen.no_malfunction_generator(),
                 malfunction_generator=None,
                 remove_agents_at_target=True,
                 random_seed=None,
                 record_steps=False,
                 close_following=True,
                 ):
        """
        Environment init.

        Parameters
        ----------
        rail_generator : function
            The rail_generator function is a function that takes the width,
            height and agents handles of a  rail environment, along with the number of times
            the env has been reset, and returns a GridTransitionMap object and a list of
            starting positions, targets, and initial orientations for agent handle.
            The rail_generator can pass a distance map in the hints or information for specific line_generators.
            Implementations can be found in flatland/envs/rail_generators.py
        line_generator : function
            The line_generator function is a function that takes the grid, the number of agents and optional hints
            and returns a list of starting positions, targets, initial orientations and speed for all agent handles.
            Implementations can be found in flatland/envs/line_generators.py
        width : int
            The width of the rail map. Potentially in the future,
            a range of widths to sample from.
        height : int
            The height of the rail map. Potentially in the future,
            a range of heights to sample from.
        number_of_agents : int
            Number of agents to spawn on the map. Potentially in the future,
            a range of number of agents to sample from.
        obs_builder_object: ObservationBuilder object
            ObservationBuilder-derived object that takes builds observation
            vectors for each agent.
        remove_agents_at_target : bool
            If remove_agents_at_target is set to true then the agents will be removed by placing to
            RailEnv.DEPOT_POSITION when the agent has reach it's target position.
        random_seed : int or None
            if None, then its ignored, else the random generators are seeded with this number to ensure
            that stochastic operations are replicable across multiple operations
        """
        super().__init__()

        if malfunction_generator_and_process_data is not None:
            print("DEPRECATED - RailEnv arg: malfunction_and_process_data - use malfunction_generator")
            self.malfunction_generator, self.malfunction_process_data = malfunction_generator_and_process_data
        elif malfunction_generator is not None:
            self.malfunction_generator = malfunction_generator
            # malfunction_process_data is not used
            # self.malfunction_generator, self.malfunction_process_data = malfunction_generator_and_process_data
            self.malfunction_process_data = self.malfunction_generator.get_process_data()
        # replace default values here because we can't use default args values because of cyclic imports
        else:
            self.malfunction_generator = mal_gen.NoMalfunctionGen()
            self.malfunction_process_data = self.malfunction_generator.get_process_data()
        
        self.number_of_agents = number_of_agents

        if rail_generator is None:
            rail_generator = rail_gen.sparse_rail_generator()
        self.rail_generator = rail_generator
        if line_generator is None:
            line_generator = line_gen.sparse_line_generator()
        self.line_generator = line_generator

        self.rail: Optional[GridTransitionMap] = None
        self.width = width
        self.height = height

        self.remove_agents_at_target = remove_agents_at_target

        self.obs_builder = obs_builder_object
        self.obs_builder.set_env(self)

        self._max_episode_steps: Optional[int] = max_episode_steps
        self._elapsed_steps = 0

        self.obs_dict = {}
        self.rewards_dict = {}
        self.dev_obs_dict = {}
        self.dev_pred_dict = {}

        self.agents: List[EnvAgent] = []
        self.num_resets = 0
        self.distance_map = DistanceMap(self.agents, self.height, self.width)

        self.action_space = [6]

        self._seed()
        if random_seed:
            self._seed(seed=random_seed)

        self.agent_positions = None

        self.run_once = [0]*(self.number_of_agents)   # Flag to check when a train has started   

        # save episode timesteps ie agent positions, orientations.  (not yet actions / observations)
        self.record_steps = record_steps  # whether to save timesteps
        # save timesteps in here: [[[row, col, dir, malfunction],...nAgents], ...nSteps]
        self.cur_episode = []
        self.list_actions = []  # save actions in here

        self.motionCheck = ac.MotionCheck()

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        random.seed(seed)
        self.random_seed = seed

        # Keep track of all the seeds in order
        if not hasattr(self, 'seed_history'):
            self.seed_history = [seed]
        if self.seed_history[-1] != seed:
            self.seed_history.append(seed)

        return [seed]

    # no more agent_handles
    def get_agent_handles(self):
        return range(self.get_num_agents())
    
    def get_num_agents(self) -> int:
        return len(self.agents)

    def add_agent(self, agent):
        """ Add static info for a single agent.
            Returns the index of the new agent.
        """
        self.agents.append(agent)
        return len(self.agents) - 1

    def reset_agents(self):
        """ Reset the agents to their starting positions
        """
        for agent in self.agents:
            agent.reset()
        self.active_agents = [i for i in range(len(self.agents))]

    def action_required(self, agent):
        """
        Check if an agent needs to provide an action

        Parameters
        ----------
        agent: RailEnvAgent
        Agent we want to check

        Returns
        -------
        True: Agent needs to provide an action
        False: Agent cannot provide an action
        """
        return agent.state == TrainState.READY_TO_DEPART or \
               ( agent.state.is_on_map_state() and agent.speed_counter.is_cell_entry )

    def reset(self, regenerate_rail: bool = True, regenerate_schedule: bool = True, *,
              random_seed: int = None) -> Tuple[Dict, Dict]:
        """
        reset(regenerate_rail, regenerate_schedule, activate_agents, random_seed)

        The method resets the rail environment

        Parameters
        ----------
        regenerate_rail : bool, optional
            regenerate the rails
        regenerate_schedule : bool, optional
            regenerate the schedule and the static agents
        random_seed : int, optional
            random seed for environment

        Returns
        -------
        observation_dict: Dict
            Dictionary with an observation for each agent
        info_dict: Dict with agent specific information

        """

        if random_seed:
            self._seed(random_seed)

        optionals = {}
        if regenerate_rail or self.rail is None:

            if "__call__" in dir(self.rail_generator):
                rail, optionals = self.rail_generator(
                    self.width, self.height, self.number_of_agents, self.num_resets, self.np_random)
            elif "generate" in dir(self.rail_generator):
                rail, optionals = self.rail_generator.generate(
                    self.width, self.height, self.number_of_agents, self.num_resets, self.np_random)
            else:
                raise ValueError("Could not invoke __call__ or generate on rail_generator")

            self.rail = rail
            self.height, self.width = self.rail.grid.shape

            # Do a new set_env call on the obs_builder to ensure
            # that obs_builder specific instantiations are made according to the
            # specifications of the current environment : like width, height, etc
            self.obs_builder.set_env(self)

        if optionals and 'distance_map' in optionals:
            self.distance_map.set(optionals['distance_map'])

        if regenerate_schedule or regenerate_rail or self.get_num_agents() == 0:
            agents_hints = None
            if optionals and 'agents_hints' in optionals:
                agents_hints = optionals['agents_hints']

            line = self.line_generator(self.rail, self.number_of_agents, agents_hints, 
                                               self.num_resets, self.np_random)
            self.agents = EnvAgent.from_line(line)

            # Reset distance map - basically initializing
            self.distance_map.reset(self.agents, self.rail)

            # NEW : Time Schedule Generation
            timetable = timetable_generator(self.agents, self.distance_map, 
                                               agents_hints, self.np_random)

            #self._max_episode_steps = 250

            for agent_i, agent in enumerate(self.agents):
                agent.earliest_departure = timetable.earliest_departures[agent_i]         
                agent.latest_arrival = timetable.latest_arrivals[agent_i]
        else:
            self.distance_map.reset(self.agents, self.rail)
        
        # Reset agents to initial states
        self.reset_agents()

        self.run_once = [0]*(self.number_of_agents)   # Flag to check when a train has started

        self.num_resets += 1
        self._elapsed_steps = 0

        # Agent positions map
        self.agent_positions = np.zeros((self.height, self.width), dtype=int) - 1
        self._update_agent_positions_map(ignore_old_positions=False)

        self.dones = dict.fromkeys(list(range(self.get_num_agents())) + ["__all__"], False)

        # Reset the state of the observation builder with the new environment
        self.obs_builder.reset()

        # Empty the episode store of agent positions
        self.cur_episode = []

        info_dict = self.get_info_dict()
        # Return the new observation vectors for each agent
        observation_dict: Dict = self._get_observations()
        if hasattr(self, "renderer") and self.renderer is not None:
            self.renderer = None
        return observation_dict, info_dict


    def _update_agent_positions_map(self, ignore_old_positions=True):
        """ Update the agent_positions array for agents that changed positions """
        for agent in self.agents:
            if not ignore_old_positions or agent.old_position != agent.position:
                if agent.position is not None:
                    self.agent_positions[agent.position] = agent.handle
                if agent.old_position is not None:
                    self.agent_positions[agent.old_position] = -1
    
    def generate_state_transition_signals(self, agent, preprocessed_action, movement_allowed, target_time):
        """ Generate State Transitions Signals used in the state machine """
        st_signals = StateTransitionSignals()
        
        # Malfunction starts when in_malfunction is set to true
        st_signals.in_malfunction = agent.malfunction_handler.in_malfunction

        # Malfunction counter complete - Malfunction ends next timestep
        st_signals.malfunction_counter_complete = agent.malfunction_handler.malfunction_counter_complete

        # Earliest departure reached - Train is allowed to move now
        st_signals.earliest_departure_reached = self._elapsed_steps >= agent.earliest_departure

        # Stop Action Given
        st_signals.stop_action_given = (preprocessed_action == RailEnvActions.STOP_MOVING)

        # Valid Movement action Given
        st_signals.valid_movement_action_given = preprocessed_action.is_moving_action() and movement_allowed

        # Target Reached
        if self._elapsed_steps >= target_time:
            # agent.target = [agent.target[0] - 1, agent.target[1]]
            st_signals.target_reached = env_utils.fast_position_equal(agent.position, agent.target)
        else:
            st_signals.target_reached = False

        # Movement conflict - Multiple trains trying to move into same cell
        # If speed counter is not in cell exit, the train can enter the cell
        st_signals.movement_conflict = (not movement_allowed) and agent.speed_counter.is_cell_exit

        return st_signals

    def _handle_end_reward(self, agent: EnvAgent) -> int:
        '''
        Handles end-of-episode reward for a particular agent.

        Parameters
        ----------
        agent : EnvAgent
        '''
        i_agent = agent.handle

        if training == 'training0':
            if i_agent != 0:
                reward = 0
                return reward
        if training == 'training1' or training == 'training1.1':
            if i_agent > 1:
                reward = 0
                return reward

        reward = None
        # agent done? (arrival_time is not None)
        if agent.state == TrainState.DONE:
            # if agent arrived earlier or on time = 0
            # if agent arrived later = -ve reward based on how late
            reward = target_reward
            i_agent = agent.handle
            self.dones[i_agent] = True
            # DELAY
            #reward = min(agent.latest_arrival - agent.arrival_time, 0)

        # Agents not done (arrival_time is None)
        else:
            # CANCELLED check (never departed)
            if (agent.state.is_off_map_state()):
                reward = -1 * self.cancellation_factor * \
                    (agent.get_travel_time_on_shortest_path(self.distance_map) + self.cancellation_time_buffer)

            # Departed but never reached
            if (agent.state.is_on_map_state()):
                reward = -0.5
                # DELAY
                #reward = agent.get_current_delay(self._elapsed_steps, self.distance_map)
        
        return reward

    def preprocess_action(self, action, agent):
        """
        Preprocess the provided action
            * Change to DO_NOTHING if illegal action
            * Block all actions when in waiting state
            * Check MOVE_LEFT/MOVE_RIGHT actions on current position else try MOVE_FORWARD
        """
        action = action_preprocessing.preprocess_raw_action(action, agent.state, agent.action_saver.saved_action)
        action = action_preprocessing.preprocess_action_when_waiting(action, agent.state)

        # Try moving actions on current position
        current_position, current_direction = agent.position, agent.direction
        if current_position is None: # Agent not added on map yet
            current_position, current_direction = agent.initial_position, agent.initial_direction
        
        action = action_preprocessing.preprocess_moving_action(action, self.rail, current_position, current_direction)

        # Check transitions, bounts for executing the action in the given position and directon
        if action.is_moving_action() and not check_valid_action(action, self.rail, current_position, current_direction):
            action = RailEnvActions.STOP_MOVING

        return action
    
    def clear_rewards_dict(self):
        """ Reset the rewards dictionary """
        self.rewards_dict = {i_agent: 0 for i_agent in range(len(self.agents))}

    def get_info_dict(self):
        """ 
        Returns dictionary of infos for all agents 
        dict_keys : action_required - 
                    malfunction - Counter value for malfunction > 0 means train is in malfunction
                    speed - Speed of the train
                    state - State from the trains's state machine
        """
        info_dict = {
            'action_required': {i: self.action_required(agent) for i, agent in enumerate(self.agents)},
            'malfunction': {
                i: agent.malfunction_handler.malfunction_down_counter for i, agent in enumerate(self.agents)
            },
            'speed': {i: agent.speed_counter.speed for i, agent in enumerate(self.agents)},
            'state': {i: agent.state for i, agent in enumerate(self.agents)}
        }
        return info_dict
    
    def update_step_rewards(self, i_agent):
        """
        Update the rewards dict for agent id i_agent for every timestep
        """
        if training == 'training0' and i_agent != 0:
            return
        elif (training == 'training1' or training == 'training1.1') and i_agent > 1:
            return

        action = self.agents[i_agent].action_saver.saved_action
        moving = self.agents[i_agent].moving
        state = self.agents[i_agent].state

        reward = None

        reward = step_penality

        if action == RailEnvActions.REVERSE:
            reward += reverse_penality
        if not moving or state == TrainState.STOPPED:
            reward += stop_penality
        
        self.rewards_dict[i_agent] += reward

    def end_of_episode_update(self, have_all_agents_ended):
        """ 
        Updates made when episode ends
        Parameters: have_all_agents_ended - Indicates if all agents have reached done state
        """
        if have_all_agents_ended or \
           ( (self._max_episode_steps is not None) and (self._elapsed_steps >= self._max_episode_steps)):

            for i_agent, agent in enumerate(self.agents):
                
                reward = self._handle_end_reward(agent)
                self.rewards_dict[i_agent] += reward
                
                #self.dones[i_agent] = True

            #self.dones["__all__"] = True

    def handle_done_state(self, agent):
        """ Any updates to agent to be made in Done state """
        if agent.state == TrainState.DONE and agent.arrival_time is None:
            agent.arrival_time = self._elapsed_steps
            if self.remove_agents_at_target:
                agent.position = None

    def step(self, action_dict_: Dict[int, RailEnvActions]):
        """
        Updates rewards for the agents at a step.
        """
        self._elapsed_steps += 1

        # Not allowed to step further once done
        if self.dones["__all__"]:
            raise Exception("Episode is done, cannot call step()")

        self.clear_rewards_dict()

        have_all_agents_ended = True # Boolean flag to check if all agents are done

        self.motionCheck = ac.MotionCheck()  # reset the motion check

        temp_transition_data = {}
        
        for agent in self.agents:
            i_agent = agent.handle

            # Build info dict
            rail, optionals = self.rail_generator(
                self.width, self.height, self.number_of_agents, self.num_resets, self.np_random)

            agent = self.agents[i_agent]

            # Calculate velocities that the agents have to mantein
            velocities = self.check_speed(optionals['agents_hints'])   # TODO variare velocità in base alla stazione da raggiungere     

            agent.speed_counter.speed = velocities[i_agent]

            # Starting time of the agent
            starting_time = optionals['agents_hints']['timetable'][i_agent][1][0]
            ending_time = optionals['agents_hints']['timetable'][i_agent][1][-1]

            agent.earliest_departure = starting_time
            agent.latest_arrival = ending_time

            agent.old_position = agent.position
            agent.old_direction = agent.direction
            # Generate malfunction
            agent.malfunction_handler.generate_malfunction(self.malfunction_generator, self.np_random)

            # Get action for the agent
            action = action_dict_.get(i_agent, RailEnvActions.DO_NOTHING)

            preprocessed_action = self.preprocess_action(action, agent)

            # Save moving actions in not already saved
            agent.action_saver.save_action_if_allowed(preprocessed_action, agent.state)

            # Train's next position can change if current stopped in a fractional speed or train is at cell's exit

            position_update_allowed = agent.speed_counter.is_cell_exit and \
                        not agent.malfunction_handler.malfunction_down_counter > 0 and \
                        not preprocessed_action == RailEnvActions.STOP_MOVING                            

            #position_update_allowed = (agent.speed_counter.is_cell_exit or agent.state == TrainState.STOPPED)

            # Calculate new position
            # Keep agent in same place if already done
            if agent.state == TrainState.DONE:
                new_position, new_direction = agent.position, agent.direction
            elif agent.state == TrainState.MALFUNCTION:
                new_position, new_direction = agent.position, agent.direction
            # Add agent to the map if not on it yet
            elif agent.position is None and agent.action_saver.is_action_saved:
                new_position = agent.initial_position
                new_direction = agent.initial_direction       
            # If movement is allowed apply saved action independent of other agents
            elif agent.action_saver.is_action_saved and position_update_allowed:
                saved_action = agent.action_saver.saved_action
                # Apply action independent of other agents and get temporary new position and direction
                new_position, new_direction  = env_utils.apply_action_independent(saved_action, 
                                                                             self.rail, 
                                                                             agent.position, 
                                                                             agent.direction)
                preprocessed_action = saved_action
            else:
                new_position, new_direction = agent.position, agent.direction

            temp_transition_data[i_agent] = env_utils.AgentTransitionData(position=new_position,
                                                                direction=new_direction,
                                                                preprocessed_action=preprocessed_action)
            
            # This is for storing and later checking for conflicts of agents trying to occupy same cell                                                    
            self.motionCheck.addAgent(i_agent, agent.position, new_position)

        # Find conflicts between trains trying to occupy same cell  TODO controlla i bug
        self.motionCheck.find_conflicts()
        
        for agent in self.agents:
            i_agent = agent.handle

            ## Update positions
            if agent.malfunction_handler.in_malfunction:
                movement_allowed = False
            else:
                movement_allowed = self.motionCheck.check_motion(i_agent, agent.position) 


            movement_inside_cell = agent.state == TrainState.STOPPED and not agent.speed_counter.is_cell_exit
            movement_allowed = movement_allowed or movement_inside_cell

            # Fetch the saved transition data
            agent_transition_data = temp_transition_data[i_agent]
            preprocessed_action = agent_transition_data.preprocessed_action

            ## Update states
            state_transition_signals = self.generate_state_transition_signals(agent, preprocessed_action, movement_allowed, ending_time)
            agent.state_machine.set_transition_signals(state_transition_signals)
            agent.state_machine.step()

            # Needed when not removing agents at target
            movement_allowed = movement_allowed and agent.state != TrainState.DONE

            # Agent is being added to map
            if agent.state.is_on_map_state():
                if agent.state_machine.previous_state.is_off_map_state():
                    agent.position = agent.initial_position
                    agent.direction = agent.initial_direction
            # Speed counter completes
                elif movement_allowed and (agent.speed_counter.is_cell_exit):
                    agent.position = agent_transition_data.position
                    agent.direction = agent_transition_data.direction
                    agent.state_machine.update_if_reached(agent.position, agent.target)

            # Off map or on map state and position should match
            env_utils.state_position_sync_check(agent.state, agent.position, agent.handle)

            # Handle done state actions, optionally remove agents
            self.handle_done_state(agent)
            
            if training == 'training0':
                if i_agent == 0:
                    have_all_agents_ended &= (agent.state == TrainState.DONE)

            elif training == 'training1' or training == 'training1.1':
                if i_agent < 2:
                    have_all_agents_ended &= (agent.state == TrainState.DONE)

            else:
                have_all_agents_ended &= (agent.state == TrainState.DONE)

            ## Update rewards
            self.update_step_rewards(i_agent)

            ## Update counters (malfunction and speed)
            agent.speed_counter.update_counter(agent.state, agent.old_position)
                                            #    agent.state_machine.previous_state)
            agent.malfunction_handler.update_counter()

            # Clear old action when starting in new cell
            if agent.speed_counter.is_cell_entry and agent.position is not None:
                agent.action_saver.clear_saved_action()
        
        # Check if episode has ended and update rewards and dones
        self.end_of_episode_update(have_all_agents_ended)

        self._update_agent_positions_map()

        return self._get_observations(), self.rewards_dict, self.dones, self.get_info_dict() 

    def record_timestep(self, dActions):
        """ 
        Record the positions and orientations of all agents in memory, in the cur_episode
        """
        list_agents_state = []
        for i_agent in range(self.get_num_agents()):
            agent = self.agents[i_agent]
            # the int cast is to avoid numpy types which may cause problems with msgpack
            # in env v2, agents may have position None, before starting
            if agent.position is None:
                pos = (0, 0)
            else:
                pos = (int(agent.position[0]), int(agent.position[1]))
            # print("pos:", pos, type(pos[0]))
            list_agents_state.append([
                    *pos, int(agent.direction), 
                    agent.malfunction_handler.malfunction_down_counter,  
                    int(agent.status),
                    int(agent.position in self.motionCheck.svDeadlocked)
                    ])

        self.cur_episode.append(list_agents_state)
        self.list_actions.append(dActions)

    def _get_observations(self):
        """
        Utility which returns the dictionary of observations for an agent with respect to environment
        """
        # print(f"_get_obs - num agents: {self.get_num_agents()} {list(range(self.get_num_agents()))}")
        self.obs_dict = self.obs_builder.get_many(list(range(self.get_num_agents())))
        return self.obs_dict

    def get_valid_directions_on_grid(self, row: int, col: int) -> List[int]:
        """
        Returns directions in which the agent can move
        """
        return Grid4Transitions.get_entry_directions(self.rail.get_full_transitions(row, col))

    def _exp_distirbution_synced(self, rate: float) -> float:
        """
        Generates sample from exponential distribution
        We need this to guarantee synchronity between different instances with same seed.
        :param rate:
        :return:
        """
        u = self.np_random.rand()
        x = - np.log(1 - u) * rate
        return x

    def _is_agent_ok(self, agent: EnvAgent) -> bool:
        """
        Check if an agent is ok, meaning it can move and is not malfuncitoinig
        Parameters
        ----------
        agent

        Returns
        -------
        True if agent is ok, False otherwise

        """
        return agent.malfunction_handler.in_malfunction
        

    def save(self, filename):
        print("DEPRECATED call to env.save() - pls call RailEnvPersister.save()")
        persistence.RailEnvPersister.save(self, filename)

    def render(self, mode="rgb_array", gl="PGL", agent_render_variant=AgentRenderVariant.ONE_STEP_BEHIND,
            show_debug=False, clear_debug_text=True, show=False,
            screen_height=600, screen_width=800,
            show_observations=False, show_predictions=False,
            show_rowcols=False, return_image=True):
        """
        This methods provides the option to render the
        environment's behavior as an image or to a window.
        Parameters
        ----------
        mode

        Returns
        -------
        Image if mode is rgb_array, opens a window otherwise
        """
        if not hasattr(self, "renderer") or self.renderer is None:
            self.initialize_renderer(mode=mode, gl=gl,  # gl="TKPILSVG",
                                    agent_render_variant=agent_render_variant,
                                    show_debug=show_debug,
                                    clear_debug_text=clear_debug_text,
                                    show=show,
                                    screen_height=screen_height,  # Adjust these parameters to fit your resolution
                                    screen_width=screen_width)
        return self.update_renderer(mode=mode, show=show, show_observations=show_observations,
                                    show_predictions=show_predictions,
                                    show_rowcols=show_rowcols, return_image=return_image)

    def initialize_renderer(self, mode, gl,
                agent_render_variant,
                show_debug,
                clear_debug_text,
                show,
                screen_height,
                screen_width):
        # Initiate the renderer
        self.renderer = RenderTool(self, gl=gl,  # gl="TKPILSVG",
                                agent_render_variant=agent_render_variant,
                                show_debug=show_debug,
                                clear_debug_text=clear_debug_text,
                                screen_height=screen_height,  # Adjust these parameters to fit your resolution
                                screen_width=screen_width)  # Adjust these parameters to fit your resolution
        self.renderer.show = show
        self.renderer.reset()

    def update_renderer(self, mode, show, show_observations, show_predictions,
                    show_rowcols, return_image):
        """
        This method updates the render.
        Parameters
        ----------
        mode

        Returns
        -------
        Image if mode is rgb_array, None otherwise
        """
        image = self.renderer.render_env(show=show, show_observations=show_observations,
                                show_predictions=show_predictions,
                                show_rowcols=show_rowcols, return_image=return_image)
        if mode == 'rgb_array':
            return image[:, :, :3]

    def close(self):
        """
        This methods closes any renderer window.
        """
        if hasattr(self, "renderer") and self.renderer is not None:
            try:
                if self.renderer.show:
                    self.renderer.close_window()
            except Exception as e:
                print("Could Not close window due to:",e)
            self.renderer = None


    def check_speed(self, agents_hints):

    # Velocity depending on the train type and on the line (Take the minimum between the two possible velocities)
        train_velocities = [0]*self.number_of_agents

        # Check for all the agents
        for i_agent, agent in enumerate(self.agents):

            # the i_agent
            agent = self.agents[i_agent]

            # Check if the agent is in the environment or not
            if agent.position != None:

                # If the agent is in the line i the max velocity is x

                # High velocity line case
                if (agent.position in av_line):  
                    train_velocities[i_agent] = min(1, agents_hints['timetable'][i_agent][2])                
                # Regional line case
                else:
                    train_velocities[i_agent] = min(1/2, agents_hints['timetable'][i_agent][2])

            # If agent is not in the environment deafault velocity is 1/2
            else:
                train_velocities[i_agent] = 1/2

        return train_velocities

# Simulation

Timer

In [33]:
from timeit import default_timer


class Timer(object):
    """
    Utility to measure times.

    TODO:
    - add "lap" method to make it easier to measure average time (+std) when measuring the same thing multiple times.
    """

    def __init__(self):
        self.total_time = 0.0
        self.start_time = 0.0
        self.end_time = 0.0

    def start(self):
        self.start_time = default_timer()

    def end(self):
        self.total_time += default_timer() - self.start_time

    def get(self):
        return self.total_time

    def get_current(self):
        return default_timer() - self.start_time

    def reset(self):
        self.__init__()

    def __repr__(self):
        return self.get()

import numpy as np
from flatland.envs.observations import TreeObsForRailEnv

def max_lt(seq, val):
    """
    Return greatest item in seq for which item < val applies.
    None is returned if seq was empty or all items in seq were >= val.
    """
    max = 0
    idx = len(seq) - 1
    while idx >= 0:
        if seq[idx] < val and seq[idx] >= 0 and seq[idx] > max:
            max = seq[idx]
        idx -= 1
    return max


def min_gt(seq, val):
    """
    Return smallest item in seq for which item > val applies.
    None is returned if seq was empty or all items in seq were >= val.
    """
    min = np.inf
    idx = len(seq) - 1
    while idx >= 0:
        if seq[idx] >= val and seq[idx] < min:
            min = seq[idx]
        idx -= 1
    return min


def norm_obs_clip(obs, clip_min=-1, clip_max=1, fixed_radius=0, normalize_to_range=False):
    """
    This function returns the difference between min and max value of an observation
    :param obs: Observation that should be normalized
    :param clip_min: min value where observation will be clipped
    :param clip_max: max value where observation will be clipped
    :return: returnes normalized and clipped observatoin
    """
    if fixed_radius > 0:
        max_obs = fixed_radius
    else:
        max_obs = max(1, max_lt(obs, 1000)) + 1

    min_obs = 0  # min(max_obs, min_gt(obs, 0))
    if normalize_to_range:
        min_obs = min_gt(obs, 0)
    if min_obs > max_obs:
        min_obs = max_obs
    if max_obs == min_obs:
        return np.clip(np.array(obs) / max_obs, clip_min, clip_max)
    norm = np.abs(max_obs - min_obs)
    return np.clip((np.array(obs) - min_obs) / norm, clip_min, clip_max)


def _split_node_into_feature_groups(node) -> (np.ndarray, np.ndarray, np.ndarray):
    data = np.zeros(6)
    distance = np.zeros(1)
    agent_data = np.zeros(4)

    data[0] = node.dist_own_target_encountered
    data[1] = node.dist_other_target_encountered
    data[2] = node.dist_other_agent_encountered
    data[3] = node.dist_potential_conflict
    data[4] = node.dist_unusable_switch
    data[5] = node.dist_to_next_branch

    distance[0] = node.dist_min_to_target

    agent_data[0] = node.num_agents_same_direction
    agent_data[1] = node.num_agents_opposite_direction
    agent_data[2] = node.num_agents_malfunctioning
    agent_data[3] = node.speed_min_fractional

    return data, distance, agent_data


def _split_subtree_into_feature_groups(node, current_tree_depth: int, max_tree_depth: int) -> (np.ndarray, np.ndarray, np.ndarray):
    if node == -np.inf:
        remaining_depth = max_tree_depth - current_tree_depth
        # reference: https://stackoverflow.com/questions/515214/total-number-of-nodes-in-a-tree-data-structure
        num_remaining_nodes = int((4 ** (remaining_depth + 1) - 1) / (4 - 1))
        return [-np.inf] * num_remaining_nodes * 6, [-np.inf] * num_remaining_nodes, [-np.inf] * num_remaining_nodes * 4

    data, distance, agent_data = _split_node_into_feature_groups(node)

    if not node.childs:
        return data, distance, agent_data

    for direction in TreeObsForRailEnv.tree_explored_actions_char:
        sub_data, sub_distance, sub_agent_data = _split_subtree_into_feature_groups(node.childs[direction], current_tree_depth + 1, max_tree_depth)
        data = np.concatenate((data, sub_data))
        distance = np.concatenate((distance, sub_distance))
        agent_data = np.concatenate((agent_data, sub_agent_data))

    return data, distance, agent_data


def split_tree_into_feature_groups(tree, max_tree_depth: int) -> (np.ndarray, np.ndarray, np.ndarray):
    """
    This function splits the tree into three difference arrays of values
    """
    data, distance, agent_data = _split_node_into_feature_groups(tree)

    for direction in TreeObsForRailEnv.tree_explored_actions_char:
        sub_data, sub_distance, sub_agent_data = _split_subtree_into_feature_groups(tree.childs[direction], 1, max_tree_depth)
        data = np.concatenate((data, sub_data))
        distance = np.concatenate((distance, sub_distance))
        agent_data = np.concatenate((agent_data, sub_agent_data))

    return data, distance, agent_data


def normalize_observation(observation, tree_depth: int, observation_radius=0):
    """
    This function normalizes the observation used by the RL algorithm
    """
    data, distance, agent_data = split_tree_into_feature_groups(observation, tree_depth)

    data = norm_obs_clip(data, fixed_radius=observation_radius)
    distance = norm_obs_clip(distance, normalize_to_range=True)
    agent_data = np.clip(agent_data, -1, 1)
    normalized_obs = np.concatenate((np.concatenate((data, distance)), agent_data))
    return normalized_obs


## RL Utils


In [39]:
import copy
import os
import pickle
import random
from collections import namedtuple, deque, Iterable

import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim

import torch.nn as nn
import torch.nn.functional as F


class DuelingQNetwork(nn.Module):
    """Dueling Q-network (https://arxiv.org/abs/1511.06581)"""

    def __init__(self, state_size, action_size, hidsize1=128, hidsize2=128):
        super(DuelingQNetwork, self).__init__()

        # value network
        self.fc1_val = nn.Linear(state_size, hidsize1)
        self.fc2_val = nn.Linear(hidsize1, hidsize2)
        self.fc4_val = nn.Linear(hidsize2, 1)

        # advantage network
        self.fc1_adv = nn.Linear(state_size, hidsize1)
        self.fc2_adv = nn.Linear(hidsize1, hidsize2)
        self.fc4_adv = nn.Linear(hidsize2, action_size)

    def forward(self, x):
        val = F.relu(self.fc1_val(x))
        val = F.relu(self.fc2_val(val))
        val = self.fc4_val(val)

        # advantage calculation
        adv = F.relu(self.fc1_adv(x))
        adv = F.relu(self.fc2_adv(adv))
        adv = self.fc4_adv(adv)

        return val + adv - adv.mean()

class Policy:
    def step(self, state, action, reward, next_state, done):
        raise NotImplementedError

    def act(self, state, eps=0.):
        raise NotImplementedError

    def save(self, filename):
        raise NotImplementedError

    def load(self, filename):
        raise NotImplementedError


class DDDQNPolicy(Policy):
    """Dueling Double DQN policy"""

    def __init__(self, state_size, action_size, parameters, evaluation_mode=False):
        self.evaluation_mode = evaluation_mode

        self.state_size = state_size
        self.action_size = action_size
        self.double_dqn = True
        self.hidsize = 1

        if not evaluation_mode:
            self.hidsize = parameters.hidden_size
            self.buffer_size = parameters.buffer_size
            self.batch_size = parameters.batch_size
            self.update_every = parameters.update_every
            self.learning_rate = parameters.learning_rate
            self.tau = parameters.tau
            self.gamma = parameters.gamma
            self.buffer_min_size = parameters.buffer_min_size

        # Device
        if parameters.use_gpu and torch.cuda.is_available():
            self.device = torch.device("cuda:0")
            # print("🐇 Using GPU")
        else:
            self.device = torch.device("cpu")
            # print("🐢 Using CPU")

        # Q-Network
        self.qnetwork_local = DuelingQNetwork(state_size, action_size, hidsize1=self.hidsize, hidsize2=self.hidsize).to(self.device)

        if not evaluation_mode:
            self.qnetwork_target = copy.deepcopy(self.qnetwork_local)
            self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.learning_rate)
            self.memory = ReplayBuffer(action_size, self.buffer_size, self.batch_size, self.device)

            self.t_step = 0
            self.loss = 0.0

    def act(self, state, eps=0.):
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def step(self, state, action, reward, next_state, done):
        assert not self.evaluation_mode, "Policy has been initialized for evaluation only."

        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)

        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % self.update_every
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > self.buffer_min_size and len(self.memory) > self.batch_size:
                self._learn()

    def _learn(self):
        experiences = self.memory.sample()
        states, actions, rewards, next_states, dones = experiences

        # Get expected Q values from local model
        q_expected = self.qnetwork_local(states).gather(1, actions)

        if self.double_dqn:
            # Double DQN
            q_best_action = self.qnetwork_local(next_states).max(1)[1]
            q_targets_next = self.qnetwork_target(next_states).gather(1, q_best_action.unsqueeze(-1))
        else:
            # DQN
            q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(-1)

        # Compute Q targets for current states
        q_targets = rewards + (self.gamma * q_targets_next * (1 - dones))

        # Compute loss
        self.loss = F.mse_loss(q_expected, q_targets)

        # Minimize the loss
        self.optimizer.zero_grad()
        self.loss.backward()
        self.optimizer.step()

        # Update target network
        self._soft_update(self.qnetwork_local, self.qnetwork_target, self.tau)

    def _soft_update(self, local_model, target_model, tau):
        # Soft update model parameters.
        # θ_target = τ*θ_local + (1 - τ)*θ_target
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

    def save(self, filename):
        torch.save(self.qnetwork_local.state_dict(), filename + ".local")
        torch.save(self.qnetwork_target.state_dict(), filename + ".target")

    def load(self, filename):
        if os.path.exists(filename + ".local"):
            self.qnetwork_local.load_state_dict(torch.load(filename + ".local"))
        if os.path.exists(filename + ".target"):
            self.qnetwork_target.load_state_dict(torch.load(filename + ".target"))

    def save_replay_buffer(self, filename):
        memory = self.memory.memory
        with open(filename, 'wb') as f:
            pickle.dump(list(memory)[-500000:], f)

    def load_replay_buffer(self, filename):
        with open(filename, 'rb') as f:
            self.memory.memory = pickle.load(f)

    def test(self):
        self.act(np.array([[0] * self.state_size]))
        self._learn()


Experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])


class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, device):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.device = device

    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = Experience(np.expand_dims(state, 0), action, reward, np.expand_dims(next_state, 0), done)
        self.memory.append(e)

    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(self.__v_stack_impr([e.state for e in experiences if e is not None])) \
            .float().to(self.device)
        actions = torch.from_numpy(self.__v_stack_impr([e.action for e in experiences if e is not None])) \
            .long().to(self.device)
        rewards = torch.from_numpy(self.__v_stack_impr([e.reward for e in experiences if e is not None])) \
            .float().to(self.device)
        next_states = torch.from_numpy(self.__v_stack_impr([e.next_state for e in experiences if e is not None])) \
            .float().to(self.device)
        dones = torch.from_numpy(self.__v_stack_impr([e.done for e in experiences if e is not None]).astype(np.uint8)) \
            .float().to(self.device)

        return states, actions, rewards, next_states, dones

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

    def __v_stack_impr(self, states):
        sub_dim = len(states[0][0]) if isinstance(states[0], Iterable) else 1
        np_states = np.reshape(np.array(states), (len(states), sub_dim))
        return np_states


## Network Parameters

In [43]:
!pip install argparse
from argparse import ArgumentParser, Namespace


parser = ArgumentParser()
parser.add_argument("-n", "--n_episodes", help="number of episodes to run", default=2500, type=int)
parser.add_argument("-t", "--training_env_config", help="training config id (eg 0 for Test_0)", default=0, type=int)
parser.add_argument("-e", "--evaluation_env_config", help="evaluation config id (eg 0 for Test_0)", default=0, type=int)
parser.add_argument("--n_evaluation_episodes", help="number of evaluation episodes", default=25, type=int)
parser.add_argument("--checkpoint_interval", help="checkpoint interval", default=100, type=int)
parser.add_argument("--eps_start", help="max exploration", default=1.0, type=float)
parser.add_argument("--eps_end", help="min exploration", default=0.01, type=float)
parser.add_argument("--eps_decay", help="exploration decay", default=0.99, type=float)
parser.add_argument("--buffer_size", help="replay buffer size", default=int(1e5), type=int)
parser.add_argument("--buffer_min_size", help="min buffer size to start training", default=0, type=int)
parser.add_argument("--restore_replay_buffer", help="replay buffer to restore", default="", type=str)
parser.add_argument("--save_replay_buffer", help="save replay buffer at each evaluation interval", default=False, type=bool)
parser.add_argument("--batch_size", help="minibatch size", default=128, type=int)
parser.add_argument("--gamma", help="discount factor", default=0.99, type=float)
parser.add_argument("--tau", help="soft update of target parameters", default=1e-3, type=float)
parser.add_argument("--learning_rate", help="learning rate", default=0.5e-4, type=float)
parser.add_argument("--hidden_size", help="hidden size (2 fc layers)", default=128, type=int)
parser.add_argument("--update_every", help="how often to update the network", default=8, type=int)
parser.add_argument("--use_gpu", help="use GPU if available", default=True, type=bool)
parser.add_argument("--num_threads", help="number of threads PyTorch can use", default=1, type=int)
parser.add_argument("--render", help="render 1 episode in 100", default=False, type=bool)
parser.add_argument("--track", help="whether to track using wandb", default=False, type=bool)
training_params = parser.parse_args()


obs_params = {
    "observation_tree_depth": 2,
    "observation_radius": 10,
    "observation_max_path_depth": 30
}

usage: ipykernel_launcher.py [-h] [-n N_EPISODES] [-t TRAINING_ENV_CONFIG]
                             [-e EVALUATION_ENV_CONFIG]
                             [--n_evaluation_episodes N_EVALUATION_EPISODES]
                             [--checkpoint_interval CHECKPOINT_INTERVAL]
                             [--eps_start EPS_START] [--eps_end EPS_END]
                             [--eps_decay EPS_DECAY]
                             [--buffer_size BUFFER_SIZE]
                             [--buffer_min_size BUFFER_MIN_SIZE]
                             [--restore_replay_buffer RESTORE_REPLAY_BUFFER]
                             [--save_replay_buffer SAVE_REPLAY_BUFFER]
                             [--batch_size BATCH_SIZE] [--gamma GAMMA]
                             [--tau TAU] [--learning_rate LEARNING_RATE]
                             [--hidden_size HIDDEN_SIZE]
                             [--update_every UPDATE_EVERY] [--use_gpu USE_GPU]
                             [--num_threads

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [45]:

# In Flatland you can use custom observation builders and predicitors
# Observation builders generate the observation needed by the controller
# Preditctors can be used to do short time prediction which can help in avoiding conflicts in the network
from flatland.envs.malfunction_generators import malfunction_from_params, MalfunctionParameters, ParamMalfunctionGen
from flatland.envs.observations import TreeObsForRailEnv, GlobalObsForRailEnv
# First of all we import the Flatland rail environment
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_env import RailEnvActions
# Import the railway generators
from flatland.utils.rendertools import RenderTool, AgentRenderVariant
# Import the agent class
from flatland.envs.step_utils.states import TrainState
from flatland.envs.predictions import ShortestPathPredictorForRailEnv
